# 验证元数据是否可以更改
最终实现设备共享规则修改

In [1]:
# metadate好像只能跟随事务
# import BigchainDB and create an object
from bigchaindb_driver import BigchainDB
bdb_root_url = 'http://192.168.1.110:9984'
bdb = BigchainDB(bdb_root_url)

In [2]:
alice_private_key='88L2BJC9eNtSWhpPwWqqsLDRGz7aBPhuRNyfsWx4QxWR'
alice_public_key='HWkENox4DM4Tp3qSfYW8igndpog9GpKFzB7Tp7yXgpBq'
bob_private_key='HwLCf9fbhm6BHTagY5aC1uVKR6sz57h7viuS8DUR9x34'
bob_public_key='3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'

In [5]:

# create a digital asset for Alice
share_device = {
    'data': {
        'owner': {
            'name':'block'
            },
        'device':{
            'type': 'parking'
            },
        },
        'description': 'share device.',
    }

metadata = {
    'isShare':1
}

# prepare the transaction with the digital asset and issue 10 tokens for Bob
prepared_token_tx = bdb.transactions.prepare(
    operation='CREATE',
    signers=bob_public_key,
    recipients=[([bob_public_key], 1)],
    asset=share_device,
    metadata=metadata
)

# fulfill and send the transaction
fulfilled_token_tx = bdb.transactions.fulfill(
    prepared_token_tx,
    private_keys=bob_private_key)
rep = bdb.transactions.send_commit(fulfilled_token_tx)
print(rep)

{'asset': {'data': {'device': {'type': 'parking'}, 'owner': {'name': 'block'}}}, 'id': 'f4d24dad7e7adff2c8aa291a910efcb68ca108f3f5377f06dee17179ea47f6ba', 'inputs': [{'fulfillment': 'pGSAICNu3miMPbgnonQOrcbGgUknVoZB1S3vKt5lGBSitv90gUDzEaJcAXAyZSEuX4Lf5kIHhlcz9H79jJaOLEIey90MRDMWEGXVuzPj8VbihxGnb0cJka0NhIoNzNEtx8TNoBEO', 'fulfills': None, 'owners_before': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm']}], 'metadata': {'isShare': 1}, 'operation': 'CREATE', 'outputs': [{'amount': '1', 'condition': {'details': {'public_key': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm', 'type': 'ed25519-sha-256'}, 'uri': 'ni:///sha-256;t1GM7Bud9-p2hvzkPDn8AN8FRSc8azG8u0BG_KLswiE?fpt=ed25519-sha-256&cost=131072'}, 'public_keys': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm']}], 'version': '2.0'}


In [11]:
# 查看该资产
# 查询
response = bdb.assets.get(search = 'block')
response

[{'data': {'owner': {'name': 'block'}, 'device': {'type': 'parking'}},
  'id': 'f4d24dad7e7adff2c8aa291a910efcb68ca108f3f5377f06dee17179ea47f6ba'}]

```
{
    'asset': {'data': {'device': {'type': 'parking'}, 'owner': {'name': 'block'}}}, 
    'id': 'f4d24dad7e7adff2c8aa291a910efcb68ca108f3f5377f06dee17179ea47f6ba', 
    'inputs': [{
                'fulfillment': 'xxx', 
                'fulfills': None, 
                'owners_before': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm']
           }], 
    'metadata': {'isShare': 1}, 
    'operation': 'CREATE', 
    'outputs': [{
                'amount': '1', 
                'condition': {
                        'details': {
                            'public_key': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm', 
                            'type': 'ed25519-sha-256'
                            }, 
                        'uri': 'ni:///shxxx'
                        }, 
                'public_keys': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm']
            }], 
    'version': '2.0'
}
```

果然metadata都是跟着事务的，所以，想修改的话，可以自交易，同时修改metadate。而创建出来的资产是永远无法修改的，哦这就是那个权限管理的根基

# GET /api/v1/assets/?search={search}
简单查询

192.168.1.110:9984/api/v1/assets/?search=block
    
```
[{"data": {"owner": {"name": "block"}, "device": {"type": parking"}},"id":"f4d24dad7e7adff2c8aa291a910efcb68ca108f3f5377f06dee17179ea47f6ba"}]
```

# 修改metadate
修改共享状态 // todo

In [3]:
# 人话：提交事务更新metadate

# 获取最新的事务
device_status = bdb.transactions.get(asset_id='f4d24dad7e7adff2c8aa291a910efcb68ca108f3f5377f06dee17179ea47f6ba')
device_status

[{'inputs': [{'owners_before': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'],
    'fulfills': None,
    'fulfillment': 'pGSAICNu3miMPbgnonQOrcbGgUknVoZB1S3vKt5lGBSitv90gUDzEaJcAXAyZSEuX4Lf5kIHhlcz9H79jJaOLEIey90MRDMWEGXVuzPj8VbihxGnb0cJka0NhIoNzNEtx8TNoBEO'}],
  'outputs': [{'public_keys': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'],
    'condition': {'details': {'type': 'ed25519-sha-256',
      'public_key': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'},
     'uri': 'ni:///sha-256;t1GM7Bud9-p2hvzkPDn8AN8FRSc8azG8u0BG_KLswiE?fpt=ed25519-sha-256&cost=131072'},
    'amount': '1'}],
  'operation': 'CREATE',
  'metadata': {'isShare': 1},
  'asset': {'data': {'owner': {'name': 'block'},
    'device': {'type': 'parking'}}},
  'version': '2.0',
  'id': 'f4d24dad7e7adff2c8aa291a910efcb68ca108f3f5377f06dee17179ea47f6ba'}]

In [4]:
# 与自己发生事务，修改metadata

new_metadata = device_status[0]['metadata']
new_metadata['isShare'] = 0
print(new_metadata) 

output_index = 0
output = device_status[0]['outputs'][output_index]
transfer_input = {
    'fulfillment': output['condition']['details'],
    'fulfills': {
        'output_index': output_index,
        'transaction_id': device_status[0]['id'],
    },
    'owners_before': output['public_keys'],
}
transfer_asset = {
     'id':device_status[0]['id'],
}
prepared_transfer_tx = bdb.transactions.prepare(
     operation='TRANSFER',
     asset=transfer_asset,
     inputs=transfer_input,
     recipients=[([bob_public_key], 1)],
     metadata=new_metadata
)
fulfilled_transfer_tx = bdb.transactions.fulfill(prepared_transfer_tx, private_keys=[bob_private_key])
fulfilled_transfer_tx

{'isShare': 0}


{'inputs': [{'owners_before': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'],
   'fulfills': {'transaction_id': 'f4d24dad7e7adff2c8aa291a910efcb68ca108f3f5377f06dee17179ea47f6ba',
    'output_index': 0},
   'fulfillment': 'pGSAICNu3miMPbgnonQOrcbGgUknVoZB1S3vKt5lGBSitv90gUDuMinYCDLrrhsKLfMXzLJU1meQmDsdQKM2refyGPgtdo1nNQo75Ff_t45Ljw1zGp_8Mcnk5msxotGUbxBBjfAP'}],
 'outputs': [{'public_keys': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'},
    'uri': 'ni:///sha-256;t1GM7Bud9-p2hvzkPDn8AN8FRSc8azG8u0BG_KLswiE?fpt=ed25519-sha-256&cost=131072'},
   'amount': '1'}],
 'operation': 'TRANSFER',
 'metadata': {'isShare': 0},
 'asset': {'id': 'f4d24dad7e7adff2c8aa291a910efcb68ca108f3f5377f06dee17179ea47f6ba'},
 'version': '2.0',
 'id': '81a900d97dd3ba69a8746a5ef5a66bb199ff84be5699fc97f8e7b45765265e4b'}

In [5]:
bdb.transactions.send_commit(fulfilled_transfer_tx)

# emmm因为网络的原因会出500的错误？？不知道为啥。。。

{'asset': {'id': 'f4d24dad7e7adff2c8aa291a910efcb68ca108f3f5377f06dee17179ea47f6ba'},
 'id': '81a900d97dd3ba69a8746a5ef5a66bb199ff84be5699fc97f8e7b45765265e4b',
 'inputs': [{'fulfillment': 'pGSAICNu3miMPbgnonQOrcbGgUknVoZB1S3vKt5lGBSitv90gUDuMinYCDLrrhsKLfMXzLJU1meQmDsdQKM2refyGPgtdo1nNQo75Ff_t45Ljw1zGp_8Mcnk5msxotGUbxBBjfAP',
   'fulfills': {'output_index': 0,
    'transaction_id': 'f4d24dad7e7adff2c8aa291a910efcb68ca108f3f5377f06dee17179ea47f6ba'},
   'owners_before': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm']}],
 'metadata': {'isShare': 0},
 'operation': 'TRANSFER',
 'outputs': [{'amount': '1',
   'condition': {'details': {'public_key': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm',
     'type': 'ed25519-sha-256'},
    'uri': 'ni:///sha-256;t1GM7Bud9-p2hvzkPDn8AN8FRSc8azG8u0BG_KLswiE?fpt=ed25519-sha-256&cost=131072'},
   'public_keys': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm']}],
 'version': '2.0'}

In [6]:
# 获取最新的事务
device_status = bdb.transactions.get(asset_id='f4d24dad7e7adff2c8aa291a910efcb68ca108f3f5377f06dee17179ea47f6ba')
device_status

[{'inputs': [{'owners_before': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'],
    'fulfills': None,
    'fulfillment': 'pGSAICNu3miMPbgnonQOrcbGgUknVoZB1S3vKt5lGBSitv90gUDzEaJcAXAyZSEuX4Lf5kIHhlcz9H79jJaOLEIey90MRDMWEGXVuzPj8VbihxGnb0cJka0NhIoNzNEtx8TNoBEO'}],
  'outputs': [{'public_keys': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'],
    'condition': {'details': {'type': 'ed25519-sha-256',
      'public_key': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'},
     'uri': 'ni:///sha-256;t1GM7Bud9-p2hvzkPDn8AN8FRSc8azG8u0BG_KLswiE?fpt=ed25519-sha-256&cost=131072'},
    'amount': '1'}],
  'operation': 'CREATE',
  'metadata': {'isShare': 1},
  'asset': {'data': {'owner': {'name': 'block'},
    'device': {'type': 'parking'}}},
  'version': '2.0',
  'id': 'f4d24dad7e7adff2c8aa291a910efcb68ca108f3f5377f06dee17179ea47f6ba'},
 {'inputs': [{'owners_before': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'],
    'fulfills': {'transaction_id': 'f4d24dad7e7adff2c8aa291a910efcb68ca108f3f5377f06

# 果然可以这么搞
自己提交事务，修改metadata，设备共享规则就放到这里用了。。。done

说起来这个地方是不是可以用 规则引擎什么的，虽然我还不知到什么是规则引擎 todo